In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import col
from pyspark.sql.functions import stddev
from pyspark.sql.functions import sum
from pyspark.sql.functions import abs, col
from pyspark.sql.functions import avg
from pyspark.sql.functions import length, col


In [2]:
spark = (SparkSession.builder
        .appName("trabalho_pratico")
        .getOrCreate())

In [3]:
data = spark.sparkContext.textFile("F:/Documentos/0.XP EDUCACAO\Módulo 2 - Bootcamp cientista de Dados/00.Trabalho prático/all_stocks_5yr.csv")

In [4]:
#Quantos registros há na planilha?

print(data.count())

619041


In [5]:
# criando df

data_df = spark.read.csv(data, header=True, inferSchema= True)
data_df.show()

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.01|14.05|13.71|1

In [6]:
# Quantos registros há na planilha para a ação da Apple (AAPL)?

Apple = data_df.filter(data_df.Name == "AAPL")

print(Apple.count())

1259


In [7]:
# Quantas empresas distintas têm registros nessa planilha?

distinto = data_df.select("Name").distinct()

distinto.count()

505

In [8]:
# Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

fechamento_maior = data_df.filter(col("close") > col("open")).count()
perc_fechamento_maior = (fechamento_maior / data.count()) * 100

print(perc_fechamento_maior)


51.52647401383753


In [9]:
# Qual o maior valor das ações da Apple (AAPL) na história?

maior_valor_apple = Apple.agg({"high": "max"}).collect()[0][0]

print(maior_valor_apple)

180.1


In [10]:
# Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

# agg para realizar a agregação dos dados nos grupos criados.
# O stddev é uma função que calcula o desvio-padrão dos valores

desvio_padrao_por_acao = data_df.groupBy("Name").agg(stddev("close").alias("desvio_padrao"))

acao_com_maior_volatilidade = desvio_padrao_por_acao.orderBy("desvio_padrao", ascending=False).first()

nome_da_acao_mais_volatil = acao_com_maior_volatilidade["Name"]
desvio_padrao_maior_volatilidade = acao_com_maior_volatilidade["desvio_padrao"]

print(nome_da_acao_mais_volatil)


PCLN


In [11]:
# Qual o dia com maior volume total de negociação da bolsa?

group_data = data_df.groupBy("date").agg(sum("volume").alias("Qtd"))

ordenacao_agrup = group_data.orderBy("Qtd", ascending=False).first()

print(ordenacao_agrup)

Row(date=datetime.date(2015, 8, 24), Qtd=4607945196)


In [12]:
# Qual a ação mais negociada da bolsa, em volume de transações? 

group_data = data_df.groupBy("Name").agg(sum("volume").alias("Qtd"))

ordenacao_agrup = group_data.orderBy("Qtd", ascending=False).first()

print(ordenacao_agrup)

Row(Name='BAC', Qtd=117884953591)


In [13]:
# Quantas ações começam com a letra “A”?

acoes_com_a = data_df.filter(col("Name").startswith("A")).count()

print(acoes_com_a)

72870


In [14]:
# Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

maior_close = data_df.filter(col("high") == col("close")).count()
perc_fechamento_maior = (maior_close / data.count()) * 100

print(perc_fechamento_maior)


1.1986282007169153


In [15]:
# Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

apple_df = data_df.filter(col("Name") == "AAPL")


# Calcular a diferença absoluta entre as colunas "open" e "close"
apple_df = apple_df.withColumn("diferenca_absoluta", abs(col("open") - col("close")))

dia_maior_variacao = apple_df.orderBy("diferenca_absoluta", ascending=False).first()

print(dia_maior_variacao)


Row(date=datetime.date(2015, 8, 24), open=94.87, high=108.8, low=92.0, close=103.12, volume=162206292, Name='AAPL', diferenca_absoluta=8.25)


In [16]:
# Em média, qual o volume diário de transações das ações da AAPL?

media = apple_df.select(avg("volume")).first()[0]

print(media)

54047899.73550437


In [17]:
# Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

distinto = data_df.select("Name").distinct()

# Calcule o comprimento dos nomes e conte as ocorrências para cada comprimento
contagem_por_comprimento = distinto.withColumn("comprimento", length(col("Name")))

contagem_por_comprimento2 = contagem_por_comprimento.groupBy("comprimento").count()

contagem_por_comprimento2.show()

+-----------+-----+
|comprimento|count|
+-----------+-----+
|          1|   10|
|          3|  323|
|          5|    5|
|          4|  117|
|          2|   50|
+-----------+-----+



In [18]:
# Qual a ação menos negociada da bolsa, em volume de transações?

group_data = data_df.groupBy("Name").agg(sum("volume").alias("Qtd"))

ordenacao_agrup = group_data.orderBy("Qtd", ascending=True).first()

print(ordenacao_agrup)

Row(Name='APTV', Qtd=92947779)
